In [ ]:
import geopyspark as gps
from pyspark import SparkContext
from geopyspark.geotrellis.constants import LayerType
from geopyspark_netcdf.datasets import Gddp
from shapely.geometry import shape

In [ ]:
sc = SparkContext(conf=gps.geopyspark_conf(appName="Crenshaw Boulevard"))

In [ ]:
uri = "s3://nasanex/NEX-GDDP/BCSD/rcp85/day/atmos/tasmin/r1i1p1/v1.0/tasmin_day_BCSD_rcp85_r1i1p1_inmcm4_2099.nc"
# uri = "/tmp/tasmin_day_BCSD_rcp85_r1i1p1_inmcm4_2099.nc"

In [ ]:
!curl 'https://raw.githubusercontent.com/jamesmcclain/SlausonAvenue/master/geojson/CA.geo.json' -o /tmp/CA.geo.json

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage

%matplotlib inline

In [ ]:
geojson = json.loads(open("/tmp/CA.geo.json").read())
ca = shape(geojson['features'][0]['geometry'])
ca

In [ ]:
snippet = Gddp.raster(uri, ca.bounds, 13)

In [ ]:
plt.imshow(snippet)

In [ ]:
M.set_center(-118.225, 33.897, 10)

In [ ]:
days = range(0, 365)

In [ ]:
raster_rdd = Gddp.rdd_of_rasters(uri, ca.bounds, days)

In [ ]:
masked_rdd = raster_rdd.mask(ca)

In [ ]:
cpt = list(Gddp.samples(uri, (-118.225, 33.897), range(0,365)).collect())
mins = [t[1] for t in raster_rdd.min_series(ca)]
maxs = [t[1] for t in raster_rdd.max_series(ca)]
means = [t[1] for t in raster_rdd.mean_series(ca)]

In [ ]:
plt.plot(days, cpt, label="Hub City")
plt.plot(days, mins, label="min")
plt.plot(days, maxs, label="max")
plt.plot(days, means, label="mean")
plt.legend(loc='best')

# Build GeoTiff from NumPy Array #

In [ ]:
import gdal

In [ ]:
snippet = scipy.ndimage.zoom(Gddp.raster(uri, ca.bounds, 33), 128, order=3)
(rows, columns) = snippet.shape

In [ ]:
wkt = """GEOGCS[\"WGS 84\",
    DATUM[\"WGS_1984\",
        SPHEROID[\"WGS 84\",6378137,298.257223563,
            AUTHORITY[\"EPSG\",\"7030\"]],
        AUTHORITY[\"EPSG",\"6326\"]],
    PRIMEM[\"Greenwich\",0],
    UNIT[\"degree\",0.0174532925199433],
    AUTHORITY[\"EPSG",\"4326\"]]"""

## Build Transform ##

http://www.gdal.org/gdal_tutorial.html

```
adfGeoTransform[0] /* top left x */
adfGeoTransform[1] /* w-e pixel resolution */
adfGeoTransform[2] /* 0 */
adfGeoTransform[3] /* top left y */
adfGeoTransform[4] /* 0 */
adfGeoTransform[5] /* n-s pixel resolution (negative value) */
```

In [ ]:
(xmin, ymin, xmax, ymax) = ca.bounds
transform = (xmin, (xmax - xmin)/columns, 0, ymax, 0, (ymin - ymax)/rows)
transform

## Dump To GeoTiff ##

https://gis.stackexchange.com/questions/58517/python-gdal-save-array-as-raster-with-projection-from-other-file

In [ ]:
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create("/tmp/ca.tif", columns, rows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform(transform)
dataset.SetProjection(wkt)
dataset.GetRasterBand(1).WriteArray(snippet)
dataset.GetRasterBand(1).SetNoDataValue(-1.0)
dataset.FlushCache()

## Display ##

In [ ]:
from geonotebook.wrappers.raster import TMSRasterData

from geopyspark.geotrellis.geotiff import get
from geopyspark.geotrellis.tms import *
from geopyspark.geotrellis.color import ColorMap

Trouble with reproject, use `gdalwarp` to reproject to WebMercator.

In [ ]:
!rm -f /tmp/ca2.tif

In [ ]:
!gdalwarp -t_srs 'EPSG:3857' /tmp/ca.tif /tmp/ca2.tif

In [ ]:
rdd = gps.geotiff.get(LayerType.SPATIAL, "/tmp/ca2.tif", max_tile_size=512, num_partitions=32)

In [ ]:
tiled_raster_layer = rdd.tile_to_layout(layout = gps.GlobalLayout(), target_crs=3857)

In [ ]:
from functools import partial
import pyproj
from shapely.geometry import shape
from shapely.ops import transform

project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:3857'))

ca2 = transform(project, ca)
ca2

In [ ]:
target = tiled_raster_layer.mask(ca2)

In [ ]:
pyramid = target.pyramid()

In [ ]:
colormap = gps.ColorMap.build(breaks=target.get_histogram(), colors='plasma')

In [ ]:
tms_server = gps.TMS.build(pyramid, display=colormap)

In [ ]:
M.add_layer(TMSRasterData(tms_server), name="gddp")

In [ ]:
M.remove_layer(M.layers[0])